In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score, f1_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter


In [2]:
df = pd.read_csv('Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv', low_memory = False)

In [3]:
#change the variable types for low memory usage
#int64 to int32,,, float64 to float32
integer = []
f = []
for i in df.columns[:-1]:
    if df[i].dtype == "int64": integer.append(i)
    else : f.append(i)

df[integer] = df[integer].astype("int32")
df[f] = df[f].astype("float32")

In [4]:
# Remove leading whitespaces in column names
df.columns =  df.columns.str.strip()
# drop nan and infinite rows
df = df[~df.isin([np.nan, np.inf, -np.inf]).any(axis=1)]

In [5]:
# analyze and decompose the "Label" column
isAttack = []
isBruteForce = []
isSQL_Injection = []
isXSS = []
for data in df["Label"]:
  if (data == "BENIGN"):
    isAttack.append(0)
    isBruteForce.append(0)
    isSQL_Injection.append(0)
    isXSS.append(0)
  elif (data[-1] == 'S'):
    isAttack.append(1)
    isBruteForce.append(0)
    isSQL_Injection.append(0)
    isXSS.append(1)
  elif (data[-1] == 'n'):
    isAttack.append(1)
    isBruteForce.append(0)
    isSQL_Injection.append(1)
    isXSS.append(0)
  elif (data[-1] == 'e'):
    isAttack.append(1)
    isBruteForce.append(1)
    isSQL_Injection.append(0)
    isXSS.append(0)

df["isAttack"] = isAttack
df["isBruteForce"] = isBruteForce
df["isSQL_Injection"] = isSQL_Injection
df["isXSS"] = isXSS

In [6]:
print(sum(df["isAttack"]))
print(sum(df["isBruteForce"]))
print(sum(df["isSQL_Injection"]))
print(sum(df["isXSS"]))
# there are three types of attack, including 1507 rows of brute force attack,
# 21 rows of SQL_Injection attack and 652 rows of XSS attack. In total, there are
# 2180 attacks in total.

2180
1507
21
652


In [7]:
# analyze and decompose the "destination port" column
print (df['Destination Port'].value_counts())

53       76545
443      35806
80       20837
123       3746
22        1035
         ...  
2600         1
59110        1
45764        1
2606         1
60146        1
Name: Destination Port, Length: 17764, dtype: int64


In [8]:
port_53 = []
port_443 = []
port_80 = []
port_123 = []
port_22 = []

for data in df["Destination Port"]:
  if (data == 53):
    port_53.append(1)
    port_443.append(0)
    port_80.append(0)
    port_123.append(0)
    port_22.append(0)
  elif (data == 443):
    port_53.append(0)
    port_443.append(1)
    port_80.append(0)
    port_123.append(0)
    port_22.append(0)
  elif (data == 80):
    port_53.append(0)
    port_443.append(0)
    port_80.append(1)
    port_123.append(0)
    port_22.append(0)
  elif (data == 123):
    port_53.append(0)
    port_443.append(0)
    port_80.append(0)
    port_123.append(1)
    port_22.append(0)
  elif (data == 22):
    port_53.append(0)
    port_443.append(0)
    port_80.append(0)
    port_123.append(0)
    port_22.append(1)
  else:
    port_53.append(0)
    port_443.append(0)
    port_80.append(0)
    port_123.append(0)
    port_22.append(0)

df["port_53"] = port_53
df["port_443"] = port_443
df["port_80"] = port_80
df["port_123"] = port_123
df["port_22"] = port_22

In [9]:
df = df.drop (columns = ["Destination Port", "Label"])

In [10]:
df.columns

Index(['Flow Duration', 'Total Fwd Packets', 'Total Backward Packets',
       'Total Length of Fwd Packets', 'Total Length of Bwd Packets',
       'Fwd Packet Length Max', 'Fwd Packet Length Min',
       'Fwd Packet Length Mean', 'Fwd Packet Length Std',
       'Bwd Packet Length Max', 'Bwd Packet Length Min',
       'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s',
       'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std',
       'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean',
       'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
       'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
       'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
       'Min Packet Length', 'Max Packet Length', 'Packet Length Mean',
       'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count',
       'SYN Flag Count', 'RST Flag Count', 'PS

In [11]:
# drop duplicate rows
df =  df.drop_duplicates(keep="first")
df.reset_index(drop=True,inplace=True)

In [12]:
#feature reduction
#dropping very high correlated features
numeric_columns = df.select_dtypes(include=[np.number])

corr_matrix = numeric_columns.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# Find features with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

print ("these columns are highly correlated   ")
print (to_drop)
##############################################################################

these columns are highly correlated   
['Total Backward Packets', 'Total Length of Bwd Packets', 'Fwd IAT Total', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Min', 'Fwd Header Length', 'Bwd Header Length', 'Fwd Packets/s', 'Packet Length Std', 'SYN Flag Count', 'ECE Flag Count', 'Average Packet Size', 'Avg Fwd Segment Size', 'Avg Bwd Segment Size', 'Fwd Header Length.1', 'Subflow Fwd Packets', 'Subflow Fwd Bytes', 'Subflow Bwd Packets', 'Subflow Bwd Bytes', 'act_data_pkt_fwd', 'Idle Mean', 'Idle Max', 'Idle Min']


In [13]:
# Drop features
df =  df.drop(to_drop, axis=1)
df.shape

(156190, 61)

In [14]:
cols = df.columns.tolist()
cols = cols[-5:] + cols[:-5]
df = df[cols]
print (df.columns)

Index(['port_53', 'port_443', 'port_80', 'port_123', 'port_22',
       'Flow Duration', 'Total Fwd Packets', 'Total Length of Fwd Packets',
       'Fwd Packet Length Max', 'Fwd Packet Length Min',
       'Fwd Packet Length Mean', 'Fwd Packet Length Std',
       'Bwd Packet Length Max', 'Bwd Packet Length Min',
       'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s',
       'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Mean', 'Fwd IAT Std', 'Bwd IAT Mean',
       'Bwd IAT Std', 'Bwd IAT Max', 'Fwd PSH Flags', 'Bwd PSH Flags',
       'Fwd URG Flags', 'Bwd URG Flags', 'Bwd Packets/s', 'Min Packet Length',
       'Max Packet Length', 'Packet Length Mean', 'Packet Length Variance',
       'FIN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count',
       'URG Flag Count', 'CWE Flag Count', 'Down/Up Ratio',
       'Fwd Avg Bytes/Bulk', 'Fwd Avg Packets/Bulk', 'Fwd Avg Bulk Rate',
       'Bwd Avg Bytes/Bulk', 'B

In [15]:
# display the current dataframe
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
df.head(1)

,port_53,port_443,port_80,port_123,port_22,Flow Duration,Total Fwd Packets,Total Length of Fwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Mean,Fwd IAT Std,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Variance,FIN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,Down/Up Ratio,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Init_Win_bytes_forward,Init_Win_bytes_backward,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Std,isAttack,isBruteForce,isSQL_Injection,isXSS
0,0,0,0,0,0,113095465,48,9668,403,0,201.416672,203.548294,923,316,417.166656,231.080948,174.012283,0.63663,1592893.875,4597264.5,16500000,3,2406286.5,5491986.0,4917177.5,7084368.5,16500000,1,0,0,0,0.21221,0,923,275.109589,54677.988281,0,0,0,1,0,0,0,0,0,0,0,0,0,571,2079,32,203985.5,575837.25,1629110,379,4277541.0,0,0,0,0


 a test on port 80 issue:

In [16]:
df2 = df.loc[df['port_80'] == 1]

In [17]:
print (sum(df2['isAttack']))

2143


In [18]:
print (df2.shape)

(19725, 61)


In [20]:
print(sum(df['port_80'])/156190)

0.12628849478199627


In [21]:
print(sum(df['port_53'])/156190)
print(sum(df['port_443'])/156190)
print(sum(df['port_123'])/156190)
print(sum(df['port_22'])/156190)

0.47615084192329854
0.2183622511044241
0.023016838465970933
0.005070747166912094


In [ ]:
features = df.iloc[:, :df.shape[1] - 4]
labels_1 = df["isAttack"]
labels_2 = df["isBruteForce"]
labels_3 = df["isSQL_Injection"]
labels_4 = df["isXSS"]

X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(features, labels_1, train_size=0.8, test_size=0.2, stratify=labels_1)
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(features, labels_2, train_size=0.8, test_size=0.2, stratify=labels_2)
X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(features, labels_3, train_size=0.8, test_size=0.2, stratify=labels_3)
X_train_4, X_test_4, y_train_4, y_test_4 = train_test_split(features, labels_4, train_size=0.8, test_size=0.2, stratify=labels_4)

In [ ]:
model_1 = LogisticRegression(solver = 'saga')
model_2 = LogisticRegression(solver='saga')
model_3 = LogisticRegression(solver='saga')
model_4 = LogisticRegression(solver='saga')

model_1.fit(X_train_1, y_train_1)
model_2.fit(X_train_2, y_train_2)
model_3.fit(X_train_3, y_train_3)
model_4.fit(X_train_4, y_train_4)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


LogisticRegression(solver='saga')

In [ ]:
print("\nCross-Validation Score1:", cross_val_score(model_1, X_train_1, y_train_1).mean(), '\n')
print("\nCross-Validation Score2:", cross_val_score(model_2, X_train_2, y_train_2).mean(), '\n')
print("\nCross-Validation Score3:", cross_val_score(model_3, X_train_3, y_train_3).mean(), '\n')
print("\nCross-Validation Score4:", cross_val_score(model_4, X_train_4, y_train_4).mean(), '\n')

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(



Cross-Validation Score1: 0.9862107058020679 



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(



Cross-Validation Score2: 0.9905083550766551 



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(



Cross-Validation Score3: 0.9997839170881029 



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(



Cross-Validation Score4: 0.9957503686521225 



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [ ]:
y_pred_1 = model_1.predict(X_test_1)
print("the first model is: ")
print(classification_report(y_test_1, y_pred_1))


y_pred_2 = model_2.predict(X_test_2)
print("the second model is: ")
print(classification_report(y_test_2, y_pred_2))


y_pred_3 = model_3.predict(X_test_3)
print("the third model is: ")
print(classification_report(y_test_3, y_pred_3))


y_pred_4 = model_4.predict(X_test_4)
print("the fourth model is: ")
print(classification_report(y_test_4, y_pred_4))

the first model is: 
              precision    recall  f1-score   support

           0       0.99      1.00      0.99     30809
           1       0.00      0.00      0.00       429

    accuracy                           0.99     31238
   macro avg       0.49      0.50      0.50     31238
weighted avg       0.97      0.99      0.98     31238

the second model is: 
              precision    recall  f1-score   support

           0       0.99      1.00      1.00     30944
           1       0.00      0.00      0.00       294

    accuracy                           0.99     31238
   macro avg       0.50      0.50      0.50     31238
weighted avg       0.98      0.99      0.99     31238

the third model is: 
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     31234
           1       0.00      0.00      0.00         4

    accuracy                           1.00     31238
   macro avg       0.50      0.50      0.50     31238
weighted a

In [ ]:
from sklearn import tree

# Create Decision Tree classifer object
clf_1 = tree.DecisionTreeClassifier()
clf_2 = tree.DecisionTreeClassifier()
clf_3 = tree.DecisionTreeClassifier()
clf_4 = tree.DecisionTreeClassifier()

# Train Decision Tree Classifer
clf_1 = clf_1.fit(X_train_1, y_train_1)
clf_2 = clf_2.fit(X_train_2, y_train_2)
clf_3 = clf_3.fit(X_train_3, y_train_3)
clf_4 = clf_4.fit(X_train_4, y_train_4)

#Predict the response for test dataset
y_pred_1 = clf_1.predict(X_test_1)
y_pred_2 = clf_2.predict(X_test_2)
y_pred_3 = clf_3.predict(X_test_3)
y_pred_4 = clf_4.predict(X_test_4)

In [ ]:
from sklearn import metrics
print("first model: ")
print("Accuracy:",metrics.accuracy_score(y_test_1, y_pred_1))

print("second model: ")
print("Accuracy:",metrics.accuracy_score(y_test_2, y_pred_2))

print("third model: ")
print("Accuracy:",metrics.accuracy_score(y_test_3, y_pred_3))

print("fourth model: ")
print("Accuracy:",metrics.accuracy_score(y_test_4, y_pred_4))

first model: 
Accuracy: 0.9991356680965491
second model: 
Accuracy: 0.9946539471156924
third model: 
Accuracy: 0.9999359754145591
fourth model: 
Accuracy: 0.9952621806773801


In [ ]:
print("the first model is: ")
print(classification_report(y_test_1, y_pred_1))

print("the second model is: ")
print(classification_report(y_test_2, y_pred_2))

print("the third model is: ")
print(classification_report(y_test_3, y_pred_3))

print("the fourth model is: ")
print(classification_report(y_test_4, y_pred_4))


the first model is: 
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     30809
           1       0.97      0.97      0.97       429

    accuracy                           1.00     31238
   macro avg       0.98      0.98      0.98     31238
weighted avg       1.00      1.00      1.00     31238

the second model is: 
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     30944
           1       0.73      0.68      0.71       294

    accuracy                           0.99     31238
   macro avg       0.86      0.84      0.85     31238
weighted avg       0.99      0.99      0.99     31238

the third model is: 
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     31234
           1       0.67      1.00      0.80         4

    accuracy                           1.00     31238
   macro avg       0.83      1.00      0.90     31238
weighted a